# UNIT 3: DATA WRANGLING
___

Many times, in order to get a dataset, you'll have to look online.  Some web sites are great and offer customizable data that can be downloaded as an CSV.  Yes!  Many, many others are not so great.  They'll have a chart on a page, but with lots of clutter and no way to download it.  And even if you do manage to download it, you sometimes have to work on the data to get it into a form that you can use.  This is affectionately called data wrangling.


# GOOD OL' COPY AND PASTE

Before getting into anything more complicated, the first thing you should always try is a simple copy and paste.  Copy the chart and paste it into a spreadsheet app.  You never know; it just might work.


# BEAUTIFUL SOUP

This is a pretty darn amazing python library that will scrape a web site for you, and then allow you to search for the exact data you're looking for.  (Scraping is the verb used to describe pulling information off of a web site.)

But in order for Beautiful Soup to do its job, it needs to know which part of the web site you want to pull data from.  If you already know HTML, this will make a lot of sense to you.  If you don't, I'll try to explain along the way.  Come on!

# HTML

Basically, web sites have 2 ways of presenting information in a chart:
1) table<br>
2) div

Both will be structured just like this... (it's kinda hard to do an empty table in jupyter, sorry...)

|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|
|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|,|&nbsp;|

In HTML, you'd use the following "tags" to lay out your table...

```
<table id='awesomeTable' class='standardTable'>

    <tr class='header' > <th> header, cell 1 </th> <th> header, cell 2 </th> </tr>
        <tbody>
    
    <tr class='data' > <td> row 1, cell 1 </td> <td>  row 1, cell 2 </td> </tr>
    <tr class='data' > <td> row 2, cell 1 </td> <td> row 2, cell 2 </td> </tr>
        </tbody>
</table>
```

If they used `<div>` instead, basically you'd replace all the tags above with `div`.

1) The table tag is the main tag for the whole chart.<br>
2) Each row in the chart is within a `tr` tag... (stands for table row).<br>
3) The header cells of the table can be included in `th` tags... (stands for table header).<br>
4) Each cell is within a `td` tag.<br>
5) Every single tag can have an id or a class that describes it.  This is very important.<br>
6) Sometimes, but not always, the non-header part of the table -- the main part of the table -- is contained within a `tbody` tag (table body).<br>

When looking through a web site, many times the table or div containing the data has a unique id or class that identifies it.  Something like "data-table".  We're going to look for this.

# LOOKING FOR WEB SITE DATA

When trying to find data on a web site using Chrome (or other browsers), there are 3 ways to do this...

1) in the "View" menu, select "Developer", and then choose "View Source"<br>
2) in the "View" menu, select "Developer", and then choose "Developer Tools"<br>
3) right click on part of the data chart and choose "Inspect"<br>

The view source option literally just shows you all of the HTML (and CSS and Javascript) that are in that web page.  Sometimes, for major web sites, it can be a bit overwhelming.  They have so much bloat that it's shocking.  Go to some web page you like and view the source, just so you get a sense of what I'm talking about.

Using developer tools sometimes works great, but sometimes it doesn't because it's hard to find what you're looking for.

Choosing to inspect the page element is often the best way.  It opens up the developer tools in your browser and takes you to **exactly the part of the HTML that contains the data that you're interested in.**

Go to one of the following pages (depending on if you're interested in world population, olympic medal counts, top movies of all time, or silver prices over time), look at the chart, and then try the inspect element trick to see where the data table is in the HTML:

https://www.worldometers.info/world-population/world-population-by-year<br>
https://en.wikipedia.org/wiki/1900_Summer_Olympics_medal_table<br>
https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/all-time<br>
https://en.wikipedia.org/wiki/Silver_as_an_investment<br>

Once the developer tools are open, look at the top panel.  This is the HTML.  Scroll around to see how the chart data is laid out in HTML.  Notice that as you move the cursor in the developer window, elements on the web page get highlighted.  Try to identify the table with an id or a name tag.  If the table has neither, you can just find all tables on the page and then check if it's the table you want.

# BEAUTIFUL SOUP AND HTML

Let's now use `Beautiful Soup` to help us get this data.  Also, we need another package here (`requests`), so make sure both `requests` and `Beautiful soup` are installed.

First, let's just grab all the HTML from this page...


In [ ]:
#run this cell!

import requests
from bs4 import BeautifulSoup as bs	# this is the Beautiful Soup package

# put the web page you chose in the quotes for the url variable
url = ""
page = requests.get(url)
soup = bs(page.content, "html.parser")

print ( soup )


Alright, this is a LOT of information, right?  Beautiful Soup has this great function (get_text() ) that will extract just the human readable part of a web site.

Try the above code, but just change the print statement to this...

    print ( soup.get_text() )
    
Still a lot of information, but it's a lot better than before.

Now, let's get the actual data we want from the page.  Beautiful Soup has a few amazing functions that can help us find things in HTML...

    find()		# finds a specific element (returns the first one if there are multiple)
    find_all()	# finds all elements that match (returns a list)

Using these 2 functions, you can search by tag (e.g., td, table, tbody, div), by id, by class, or even by string contained in the tag.

    find('table')		finds the 1st table on the page (finds the first element with a tag of "table")
    find_all('table')		finds all of the tables and stores them in a list
    find(id='link2')		finds the element with an id of "link2"
    find(class_="stats")	finds the 1st element with a class of "stats"


In [ ]:
url = "https://en.wikipedia.org/wiki/1900_Summer_Olympics_medal_table"
page = requests.get(url)
soup = bs(page.content, "html.parser") # all of the HTML on the entire page

# looking for a specific table with an id
table = soup.find(id="")

# looking for all tables (this can return a list of multiple tables, check to see if that's the case and index the list so there's only 1 table)
table = soup.find_all("table")

#you could try print(table[0])

If you want, go ahead and print the text within the table.  But, we actually need the table rows (tr) that are contained in the table.  So, add these lines...

In [ ]:
tbody = table[0].find("tbody") #it's table[0] since there were multiple tables in the link...
#...and we can only search through one
rows = tbody.find_all("tr") # notice that this is find_all, so it returns all the rows

The rows variable now contains a list of all the rows in the data table that we're interested in.  Try to create a for loop and view all of the table rows.  Make sure that the data matches what you see on the web site.

To find out more about Beautiful Soup functions and capabilities, go to this web site...

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

# WRITING TO A CSV

It's great to have this data, but it'd be even better to have a CSV file of this data.  Install the CSV package and let's do exactly that.  Include this import line at the top of your code and then add the rest of this code to your file.  Study this new code to ensure it all makes sense.

In [ ]:
#study, then run this

import csv # imports the CSV package

data = [] # store all of the table data in data list
for tr in rows: # for each row in the rows list
    cells = tr.find_all("td") # get the cells (td) in each row of the table
    temp = []
    for td in cells: # for each of the cells in the row
        temp.append( td.get_text() )
    data.append(temp) # store row of data in data variable

csvfile = "scraped_data.csv" # choose a filename for your new CSV file

with open(csvfile, "w") as output: # the "w" means write to this new file
    writer = csv.writer(output, lineterminator='\n')
    for val in data: # for each row in the table data
        writer.writerow(val) # write this row to the CSV


When you're done, there should be a CSV file in the same folder as your python file (you can open in it in the jupyter notebook windows), and the CSV file should contain all of the data from this web site.  Confirm that this is the case.
